In [4]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns

#downloading from google sheets
import gspread 
from oauth2client.service_account import ServiceAccountCredentials

#preparing to send to Google Sheets
from pandas.io.json import json_normalize

%matplotlib inline

In [7]:
# Set the file paths
whale_data = Path("./Resources/whale_returns.csv")
algo_data = Path("./Resources/algo_returns.csv")
sp500_data = Path("./Resources/sp500_history.csv")

# aapl_data = Path("./Resources/aapl_historical.csv")
# cost_data = Path("./Resources/cost_historical.csv")
# goog_data = Path("./Resources/goog_historical.csv")

# Read the CSVs and set the `date` column as a datetime index to the DataFrame
whale_df = pd.read_csv(whale_data, index_col="Date", \
    infer_datetime_format=True, parse_dates=True)
algo_df = pd.read_csv(algo_data, index_col="Date", \
    infer_datetime_format=True, parse_dates=True)
sp500_df = pd.read_csv(sp500_data, index_col="Date", \
    infer_datetime_format=True, parse_dates=True)

# Sort datetime index in ascending order (past to present) 
whale_df.sort_index(inplace=True)
algo_df.sort_index(inplace=True)
sp500_df.sort_index(inplace=True)

# aapl_df = pd.read_csv(aapl_data, index_col="Trade DATE", infer_datetime_format=True, parse_dates=True)
# cost_df = pd.read_csv(cost_data, index_col="Trade DATE", infer_datetime_format=True, parse_dates=True)
# goog_df = pd.read_csv(goog_data, index_col="Trade DATE", infer_datetime_format=True, parse_dates=True)

# Display a few rows
whale_df.head()

,SOROS FUND MANAGEMENT LLC,PAULSON & CO.INC.,TIGER GLOBAL MANAGEMENT LLC,BERKSHIRE HATHAWAY INC
Date,,,,
2015-03-02,NaN,NaN,NaN,NaN
2015-03-03,-0.001266,-0.004981,-0.000496,-0.006569
2015-03-04,0.002230,0.003241,-0.002534,0.004213
2015-03-05,0.004016,0.004076,0.002355,0.006726
2015-03-06,-0.007905,-0.003574,-0.008481,-0.013098


In [67]:
# dropping nulls
whale_df.dropna(inplace=True)
algo_df.dropna(inplace=True)
sp500_df.dropna(inplace=True)

In [69]:
# remove non-numeric symbols ($ and ,) , then convert to float
# sp500_df
sp500_df['Close'] = sp500_df['Close'].str.replace(',', '')
sp500_df['Close'] = sp500_df['Close'].str.replace('$', '')
sp500_df['Close'] = sp500_df['Close'].astype(float)

/var/folders/wc/dzcyrwqx2r18c1qvj6l_bc540000gn/T/ipykernel_45301/2665572392.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  sp500_df['Close'] = sp500_df['Close'].str.replace('$', '')


In [74]:
# create daily returns from the closing price of sp500
sp500_df_dr = sp500_df.pct_change()
sp500_df_dr.rename(columns={"Close": "S&P TSX"}, inplace=True)
sp500_df_dr.sort_index(inplace=True)
sp500_df_dr.head()

,S&P TSX
Date,
2012-10-01,NaN
2012-10-02,0.000872
2012-10-03,0.003624
2012-10-04,0.007174
2012-10-05,-0.000322


In [77]:
# Use the `concat` function to combine the DataFrames by matching indexes (or in this case `date`)
combined_df_list=[whale_df,algo_df,sp500_df_dr]
combined_df = pd.concat(combined_df_list, axis=1, join="inner", sort=True)
combined_df.head()

,SOROS FUND MANAGEMENT LLC,PAULSON & CO.INC.,TIGER GLOBAL MANAGEMENT LLC,BERKSHIRE HATHAWAY INC,Algo 1,Algo 2,S&P TSX
Date,,,,,,,
2015-03-03,-0.001266,-0.004981,-0.000496,-0.006569,-0.001942,-0.000949,-0.004539
2015-03-04,0.002230,0.003241,-0.002534,0.004213,-0.008589,0.002416,-0.004389
2015-03-05,0.004016,0.004076,0.002355,0.006726,-0.000955,0.004323,0.001196
2015-03-06,-0.007905,-0.003574,-0.008481,-0.013098,-0.004957,-0.011460,-0.014174
2015-03-09,0.000582,0.004225,0.005843,-0.001652,-0.005447,0.001303,0.003944


In [9]:
#part 3 - Create a custom Portfolio
# Connect to our service account
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./sa_cred.json', scope)
gc = gspread.authorize(credentials)
##Get candidate data sheet from Google Drive
spreadsheet_key = '1dQNs-d9fFF4j6sZQjORiP1BhkZbmLnzHxW6toTA6NQU'
book = gc.open_by_key(spreadsheet_key)

In [11]:
#connect to nike worksheet
nike_ws = book.worksheet("Nike")
nike_table = nike_ws.get_all_values()
##Convert table data into a dataframe
nike_df = pd.DataFrame(nike_table[1:], columns=nike_table[0])
##Only keep columns we need
nike_df = nike_df[['Date', 'Close']]
##Convert number strings to floats and ints
nike_df = nike_df.apply(pd.to_numeric, errors='ignore')
##Convert date strings to datetime format
nike_df['Date'] = pd.to_datetime(nike_df['Date'],infer_datetime_format=True)
nike_df.head()

,Date,Close
0,2000-01-03 16:00:00,6.02
1,2000-01-04 16:00:00,5.69
2,2000-01-05 16:00:00,6.02
3,2000-01-06 16:00:00,5.98
4,2000-01-07 16:00:00,5.98


In [89]:
#connect to Amazon worksheet
amzn_ws = book.worksheet("Amazon")
amzn_table = amzn_ws.get_all_values()
##Convert table data into a dataframe
amzn_df = pd.DataFrame(amzn_table[1:], columns=amzn_table[0])
##Only keep columns we need
amzn_df = amzn_df[['Date', 'Close']]
##Convert number strings to floats and ints
amzn_df = amzn_df.apply(pd.to_numeric, errors='ignore')
##Convert date strings to datetime format
amzn_df['Date'] = pd.to_datetime(amzn_df['Date'],infer_datetime_format=True)
amzn_df.head()

,Date,Close
0,2012-01-03 16:00:00,8.95
1,2012-01-04 16:00:00,8.88
2,2012-01-05 16:00:00,8.88
3,2012-01-06 16:00:00,9.13
4,2012-01-09 16:00:00,8.93


In [90]:
#connect to AMD worksheet
amd_ws = book.worksheet("AMD")
amd_table = amd_ws.get_all_values()
##Convert table data into a dataframe
amd_df = pd.DataFrame(amd_table[1:], columns=amd_table[0])
##Only keep columns we need
amd_df = amd_df[['Date', 'Close']]
##Convert number strings to floats and ints
amd_df = amd_df.apply(pd.to_numeric, errors='ignore')
##Convert date strings to datetime format
amd_df['Date'] = pd.to_datetime(amd_df['Date'],infer_datetime_format=True)
amd_df.head()

,Date,Close
0,2012-01-03 16:00:00,5.48
1,2012-01-04 16:00:00,5.46
2,2012-01-05 16:00:00,5.46
3,2012-01-06 16:00:00,5.43
4,2012-01-09 16:00:00,5.59
